<h1>Dataset Cleaning and Preparation</h1>
<hr/>
<em>Predicting Propensity to Purchase a Vehicle | Motus Hackathon Competition 2025</em>
<author>Sifiso Rimana<author> <br/> 
Student Number: 222138688 <br/>
Email: 222138688@student.uj.ac.za / rimanasifiso@gmail.com <br/>


In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('ggplot')
import re

In [2]:
# constants
DATA_URL = "https://www.mxhackathon.co.za/docs/TrainData.csv"
EMAIL_REGEX = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
DOMAIN_REGEX = r'^@[a-zA-Z0-9][a-zA-Z0-9.-]*\.[a-zA-Z]{2,}$'

In [3]:
train_df = pd.read_csv(DATA_URL)
train_df.head()

,CustomerID,LeadID,DTLeadCreated,DTLeadAllocated,Dealer,LeadSource,LeadType,Seek,InterestMake,InterestModel,...,OBSEmail,Domain,CellPrefix,CellPhoneNoLength,HourOfEnquiry,DayOfEnquiry,InFinanceProcessSystemApp,FinanceApplied,FinanceApproved,VehicleSold
0,D3B8E20E-F798-4CA9-94EE-FF6D0E7E99F2,11187465,2025-02-15 14:34:00,2025-02-17 11:47:00,Motus Chery Germiston,DEALER FACEBOOK,Facebook,New,Chery,Tiggo 4 Pro 1.5 LIT MIT,...,XXXXXXXXXXXXXXXXXXX5@gmail.com,@gmail.com,61,10,14,15,0,0,0,0
1,372AB00B-6A1C-45D1-8D29-29287C627D3B,11187550,2025-02-15 14:50:00,2025-02-15 14:50:00,Renault Northcliff - 70045,RR-Dealer Facebook-Renault,Facebook,New,Renault,Triber,...,XXXXXXXX@gmail.com,@gmail.com,83,10,14,15,0,0,0,0
2,74262064-94E9-4E7E-9C01-CA8405243929,11187564,2025-02-15 14:51:00,2025-02-17 09:01:00,Durban Multifranchise - 70111,KIA-CCPPG-SONET,Other,New,KIA,Sonet,...,XXXXXXXXX39@gmail.com,@gmail.com,79,10,14,15,0,0,0,0
3,F4D65935-310B-455D-B414-1759C3B807F7,11187583,2025-02-15 14:55:00,2025-02-15 14:57:00,Lindsay Saker VW Germiston,Group Website VW: Specials,Website,New,Volkswagen,Polo Vivo,...,XXXXXXXXXX@gmail.com,@gmail.com,71,10,14,15,1,0,0,1
4,A732E82D-B4BE-438C-9690-07AB3EFD9CD4,11187615,2025-02-15 15:04:00,2025-02-15 15:04:00,Durban Multifranchise - 70111,TikTok,TikTok,New,Renault,Kwid,...,XXXXXXXXXX12@gmail.com,@gmail.com,71,10,15,15,0,0,0,0


In [4]:
# dropping fields which are not to be used in modelling
train_df = train_df.drop(['InFinanceProcessSystemApp', 'FinanceApplied', 'FinanceApproved'], axis=1)
train_df.columns = [col.lower() for col in train_df.columns]
df = train_df.copy()
df.head()

,customerid,leadid,dtleadcreated,dtleadallocated,dealer,leadsource,leadtype,seek,interestmake,interestmodel,obsfullname,obsemail,domain,cellprefix,cellphonenolength,hourofenquiry,dayofenquiry,vehiclesold
0,D3B8E20E-F798-4CA9-94EE-FF6D0E7E99F2,11187465,2025-02-15 14:34:00,2025-02-17 11:47:00,Motus Chery Germiston,DEALER FACEBOOK,Facebook,New,Chery,Tiggo 4 Pro 1.5 LIT MIT,XXXXXXXXXX XXXXXXXXX,XXXXXXXXXXXXXXXXXXX5@gmail.com,@gmail.com,61,10,14,15,0
1,372AB00B-6A1C-45D1-8D29-29287C627D3B,11187550,2025-02-15 14:50:00,2025-02-15 14:50:00,Renault Northcliff - 70045,RR-Dealer Facebook-Renault,Facebook,New,Renault,Triber,XXXXX XXXXXX,XXXXXXXX@gmail.com,@gmail.com,83,10,14,15,0
2,74262064-94E9-4E7E-9C01-CA8405243929,11187564,2025-02-15 14:51:00,2025-02-17 09:01:00,Durban Multifranchise - 70111,KIA-CCPPG-SONET,Other,New,KIA,Sonet,XXXXXX XXXXX,XXXXXXXXX39@gmail.com,@gmail.com,79,10,14,15,0
3,F4D65935-310B-455D-B414-1759C3B807F7,11187583,2025-02-15 14:55:00,2025-02-15 14:57:00,Lindsay Saker VW Germiston,Group Website VW: Specials,Website,New,Volkswagen,Polo Vivo,XXXXX-XXX XX XXXXX,XXXXXXXXXX@gmail.com,@gmail.com,71,10,14,15,1
4,A732E82D-B4BE-438C-9690-07AB3EFD9CD4,11187615,2025-02-15 15:04:00,2025-02-15 15:04:00,Durban Multifranchise - 70111,TikTok,TikTok,New,Renault,Kwid,XXXXX XXXXXXXX XXXXX X/X,XXXXXXXXXX12@gmail.com,@gmail.com,71,10,15,15,0


# 1. Cleaning and Preparation

## 1.1 Email Analysis

- **`OBSEmail`** and **`domain`** $\rightarrow$ **`is_valid_email`, `is_email_provided`**

Email is deemed provided is when:
1) Email is not empty (that is, email field is not empty)

Email is valid when:
1) Email is deemed provided
2) Email passes the regular expression match
3) The domain associated with the email is valid 

The domain associated with the email is valid when
1) The domain passes the regular expression 
2) The domain TLD is valid
3) The domain DNS is valid

We could also capture when emails with free providers by `is_personal_email`. The email, in this case, is deemed a personal email if:
1) The domain is one of $\{\text{gmail.com}, \text{yahoo.com}, \text{hotmail.com}, \text{outlook.com}, \text{icloud.com}\}$ OR
2) The domain is closer to one of the domains in 1), (this could be a typo). We will use the Levenshtein distance to measure closeness, please refer to this [Wikipedia article](https://en.wikipedia.org/wiki/Levenshtein_distance) for more information about the Levenshtein distance measurement.



In [73]:
import tldextract
import dns.resolver
from Levenshtein import distance
free_providers = {"gmail.com", "yahoo.com", "hotmail.com", "outlook.com", "icloud.com"}

# function to check if domain DNS is valid
def has_mx_records(domain):
    domain_clean = domain.lstrip('@')
    try:
        mx_records = dns.resolver.resolve(domain_clean, 'MX')
        return len(mx_records) > 0
    except:
        return False

# function to detect typo distances
def typo_distance_of_popular(domain):
    return min([distance(domain.lstrip('@').lower(), provider) for provider in free_providers])
    
# function to check if domain overall is valid
def is_valid_domain(domain):
    if not isinstance(domain, str): return 0
    domain = domain.strip()
    if 0 < typo_distance_of_popular(domain) <= 4: return 0  
    return int(
        (bool(re.fullmatch(DOMAIN_REGEX, domain, flags=re.IGNORECASE)) and # check if domain passes the regex match
        bool(tldextract.extract(domain.lstrip('@').lower()).suffix))   # check if TLD is valid 
    ) 

# function to check if the email is provided by free providers
def is_free_email_provider(domain, max_distance=4):
    if not isinstance(domain, str): return 0
    return int(0 <= typo_distance_of_popular(domain.strip()) <= max_distance)
# function to check if email is valid
def is_valid_email(email: str, domain:str, is_email_provided: int):
    return (is_email_provided and 
            is_valid_domain(domain) and 
            int(bool(re.fullmatch(EMAIL_REGEX, email, flags=re.IGNORECASE))))



In [76]:
df['is_free_email_provider'] = df['domain'].apply(is_free_email_provider)

## 1.2 Cellphone Analysis

- A valid `cellprefix` must have two digits
- A valid cellphone number must 10 digits AND a valid `cellprefix`
- A personal phone has `cellprefix` beginning with 6 or 7 or 8

In [ ]:
def is_valid_cell(cell_prefix: str, cell_length: int):
    return int(isinstance(cell_length, int) and cell_length == 10 and len(cell_prefix) == 2)

In [84]:
# for _, row in df.iterrows():
#     print(f"{row['cellprefix']}\t {row['cellphonenolength']} \t {'✅' if is_valid_cell(row['cellprefix'], row['cellphonenolength']) else '❌'}")

In [86]:
def is_personal_cell(cellprefix):
    return int(len(cellprefix) == 2 and (6 <= int(cellprefix[0]) <= 8))

In [87]:
df['is_personal_cell'] = df['cellprefix'].apply(is_personal_cell)

In [88]:
df['is_valid_cell']=df.apply(lambda row: is_valid_cell(row['cellprefix'], row['cellphonenolength']), axis=1)

## 1.3 `HourOfEnquiry` and `DayOfEnquiry`

In [ ]:
tod_bins = [0, 4, 7, 12, 17, 21, 24]
tod_labels = ['night', 'dawn', 'morning', 'afternoon', 'evening', 'night']

df['time_of_day'] = pd.cut(df['hourofenquiry'], bins=tod_bins, labels=tod_labels, right=False,ordered=False)

In [ ]:
dom_bins = [1, 15, 25, 30]
dom_labels = ['Beginning of Month', 'Mid Month', 'Month End']

df['month_period'] = pd.cut(df['dayofenquiry'], bins=dom_bins, labels=dom_labels, right=False,ordered=False)

## 1.4 `InterestMake` and `InterestModel`

In [94]:
df['interestmake'] = df['interestmake'].fillna('NO INTEREST MAKE SELECTED')
df['interestmodel'] = df['interestmodel'].fillna('NO INTEREST MODEL SELECTED')
df['interest_car'] = df['interestmake'].str.lower() + " " + df['interestmodel'].str.lower()

In [95]:
df[['interest_car', 'customerid']]

,interest_car,customerid
0,chery tiggo 4 pro 1.5 lit mit,D3B8E20E-F798-4CA9-94EE-FF6D0E7E99F2
1,renault triber,372AB00B-6A1C-45D1-8D29-29287C627D3B
2,kia sonet,74262064-94E9-4E7E-9C01-CA8405243929
3,volkswagen polo vivo,F4D65935-310B-455D-B414-1759C3B807F7
4,renault kwid,A732E82D-B4BE-438C-9690-07AB3EFD9CD4
...,...,...
73281,toyota corolla quest 1.8,3435978B-3B9E-49E7-A5B4-DE3143011D85
73282,renault kwid,0C4DE9EB-5332-4EDE-8226-4C6E549444C9
73283,honda elevate,BB600436-0F7C-432B-A583-B2E8CFDDAF6E
73284,suzuki swift,8DC4EE79-B4DC-4CE4-A20B-86ED534A7328


In [107]:
from sklearn.feature_extraction.text import TfidfVectorizer

interest_car_vectorizer = TfidfVectorizer(max_features=2000, stop_words='english', ngram_range=(1, 2))
interest_car_vectorizer.fit_transform(df['interest_car'])

<73286x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 377235 stored elements in Compressed Sparse Row format>

In [109]:
# Get top weighted terms
tfidf_weights = dict(zip(interest_car_vectorizer.get_feature_names_out(), interest_car_vectorizer.idf_))
sorted_weights = sorted(tfidf_weights.items(), key=lambda x: x[1], reverse=False)
print("Top discriminative terms:\n", sorted_weights[:10])

Top discriminative terms:
 [('renault', 2.237508831324985), ('kwid', 3.076267303453618), ('volkswagen', 3.1377492023114724), ('renault kwid', 3.163417180487987), ('toyota', 3.307194057846465), ('selected', 3.3139336473483283), ('kia', 3.3397965552870037), ('model', 3.3605456720001694), ('model selected', 3.37071869971322), ('polo', 3.4737124270987385)]


In [115]:
df['interestmake'].str.lower().value_counts()[:100]

interestmake
renault                 18784
volkswagen               8030
toyota                   7239
kia                      5500
hyundai                  4087
                        ...  
xtrail                      3
nissan navara               3
fiat 500                    2
izusu                       2
kia sonet 1.5 ex cvt        2
Name: count, Length: 100, dtype: int64

In [120]:
for c in df['interestmodel'].str.lower().unique():
    print(c)

tiggo 4 pro 1.5 lit mit
triber
sonet
polo vivo
kwid
grand i10 1.2 premium a/t
i20 1.2 motion
urban cruiser 1.5 xr manual
tiggo 4 pro
rio hatch 1.2 ls
no interest model selected
1.0 motion hatch
creta
grand i10
polo sedan 1.4 comfortline
sportage
np200
c-class
kwid dynamique
clio
corolla quest
kiger
c-hr 1.2t luxury
kwid 1.0 zen
corolla quest 1.8 auto
polo
agya 1.0
hatch 1.2 ls
amaze
starlet 1.5 xi
seltos
crafter
tiggo 4 pro 1.5 lit manual
kwid 1.0 climber
seltos 1.6 ex
vitara
clio v
x-pander
california 6.1
figo hatch 1.5 ambiente
magnite
picanto
xpander
the hilux 2.8 gd-6 rb raider dc automatic from only r7 899 per month 
hilux
cx-60
polo vivo life
kwid climber
kwid 1.0 dynamique zen 5dr amt
eclipse cross
diesel sesfikile
ses'fikile
q3
exter
grand i10 sedan
renault
ranger
koleos
tiguan
2023 kwid zen
q2
the hilux 2.8 gd-6 rb raider dc automatic from only r7 899 pm undefined
zs
a4 sedan 35 tfsi
t-roc
renualt kiger
ignis
polo vivo 5-door 1.4
discovery
sorento
tiggo 7 pro max executive
tig

## Final Function for Data Preparation

In [ ]:
def prepare_df(df: pd.DataFrame = train_df) -> pd.DataFrame:
    """
    Cleans and prepares the dataframe into a format ready for transformation
    """
    df['is_email_provided'] = df['obsemail'].apply(lambda email: int(isinstance(email, str) and len(email) > 0))
    df['is_valid_email'] = df.apply(lambda row: is_valid_email(email=row['obsemail'], 
                                                                domain=row['domain'], 
                                                                is_email_provided=row['is_email_provided']), axis=1)
    df['is_personal_email'] = df['domain'].apply(is_free_email_provider)
    df['is_personal_cell'] = df['cellprefix'].apply(is_personal_cell)
    df['is_valid_cell']=df.apply(lambda row: is_valid_cell(row['cellprefix'], row['cellphonenolength']), axis=1)
    df['time_of_day'] = pd.cut(df['hourofenquiry'], bins=tod_bins, labels=tod_labels, right=False,ordered=False)
    df['month_period'] = pd.cut(df['dayofenquiry'], bins=dom_bins, labels=dom_labels, right=False,ordered=False)

## 

In [ ]:
#